# Prepare Input Data for Cargill Supply Chain Analysis

In [132]:
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
import os
from os import listdir
import qgrid
from tqdm import tqdm

## Data Location

### Inputs 

In [3]:
# Updatable locations
root = r'c:\Users\Samantha.Kuzma\Cargill Inc\Water WRI Cargill - Documents\General'
version = "v2_updatable"
run = "run_202011"

# --------------------------------------------------------------------------------------------------------------
# Main path folders
input_folder = os.path.join(root, "Watershed Validation Data", "Inputs")
analysis_folder = os.path.join(root, "Watershed Validation Data", "Analysis")
final_folder = os.path.join(root, "Watershed Validation Data", "Final")


# --------------------------------------------------------------------------------------------------------------
# Geospatial Data
GDBIN =  os.path.join(root, "WRI_Data", "analysis", "filter_datasets.gdb") # contains hydrobasins
GDBSEL = os.path.join(root, "WRI_Data", "analysis", "filter_selections.gdb") # US Counties
GDBTEMP =  os.path.join(root, "WRI_Data", "analysis", "hybas6_adm1.shp")
# GDBTEMP2 =  os.path.join(root, "WRI_Data", "analysis", "v2_hybas6_adm1.shp")
# --------------------------------------------------------------------------------------------------------------
# Lookup Tables (NEED TO UPDATE IF ADDING NEW BUSINESS DATA)
crop_lookup_path = os.path.join(input_folder, "_update_master_crop_list.csv") # matches Cargill crops to SPAM crops
country_lookup_path = os.path.join(input_folder, "_update_country_iso_lookup.csv")
aqueduct_db_path = os.path.join(root, "WRI_Data", "sources", "WRI", "y2019m07d11_aqueduct30_annual_v01.csv")

# --------------------------------------------------------------------------------------------------------------
# IFPRI Crop Data
ifpri_root =  r'c:\Users\Samantha.Kuzma\OneDrive - World Resources Institute\Cargill\Data\sources\ifpri'
ifpri_prod = os.path.join(ifpri_root, "spam2010v2r0_global_prod.geotiff") # Crop production
ifpri_harv = os.path.join(ifpri_root, "spam2010v1r1_global_harv_area.geotiff") # Crop production

ifpri_gdb = os.path.join(ifpri_root, 'hybas6adm1.gdb')

# Create union of watersheds and countries

In [59]:
# # Hydrobasin layer
# hy6 = gpd.read_file(GDBIN, layer='hybas6')
# # Read in data for states
# adm0 = gpd.read_file(GDBIN, layer='gadm_adm0')

In [45]:
# Clean up union file
poly = gpd.read_file(GDBTEMP)
# # Filter important columns
# poly2 = poly.filter(['pfaf_id', "GID_1",  'geometry'])
# # # Replace nulls
# poly2['pfaf_id'].replace(0, -9999, inplace = True)
# poly2['GID_1'].replace(np.nan, '-9999', inplace = True)
# # # Create unique ID
# string_id = 'id_{:s}_{:s}'.format
# poly2['string_id'] = poly2.apply(lambda x: string_id(str(x['pfaf_id']), x['GID_1']), axis=1)
# Export shapefile
# poly2.to_file(driver = 'ESRI Shapefile', filename = GDBTEMP2)

In [3]:
# Hydrobasin 6 / Admin Level 1 Union
shape = gpd.read_file(GDBIN, layer='hybas6_adm1')

## Functions

In [4]:
def find_ifpri_files(ifpri_type):

    all_files = listdir(ifpri_type)
    all_tifs = [x for x in all_files if (".ovr" not in x) & ('8C0XTN2' not in x)]
    all_irr = [x for x in all_tifs if ("_i.tif" in x.lower())]
    all_all = [x for x in all_tifs if ("_a.tif" in x.lower())]
    # Count how many files are in both lists (should be 42)
    print("Irrigated:", len(all_irr), "All:", len(all_all))


    # 4. 
    return all_irr, all_all

In [26]:
ifpri_type = ifpri_prod
ifpri_type_name = "production"
out_format = "crop_{:s}_{:s}_{:s}.csv".format
# 1. Find all crop files within IFPRI folder
all_files = listdir(ifpri_type)
all_tifs = [x for x in all_files if (".ovr" not in x) & ('8C0XTN2' not in x)]
# 2. Loop through irrigation types
for irr_typ in ["_a.tif"]:
    irr_label = irr_typ[1:2]
    tifs = [x for x in all_tifs if (irr_typ in x.lower())]
#     # 2. Create list of crop names from files
#     crops = sorted([x[-10:-6] for x in tifs])
    cargill_crops = ['barl', 'bean', 'cass', 'cnut', 'coco', 'cott', 'lent',
                     'maiz', 'ocer', 'ofib', 'oilp', 'rape', 'rice', 'sorg',
                     'soyb', 'sunf', 'whea', 'sugc', 'opul']
    cargill_crops = ['maiz']
    crops = cargill_crops
    print("Irrigation Type:", irr_label, "\nLength:", len(tifs))
    # 3. Create master dataframe to hold all sums
    df_sums = shape.filter(['string_id'])
    
    # 4. Loop through crops, run zonal statistics
    for c in range(0, len(crops)):
        crp_name = crops[c] # crop name
        crp = tifs[c]    # file name
        crp_tif = os.path.join(ifpri_type, crp) # file path
        print(crp_name)
        # 5. Run zonal statistic
        sums = zonal_stats(shape, crp_tif, stats="sum")
        # 6. Add data to master dataframe
        #Turn raster stats into dataframe
        df_temp = pd.DataFrame(sums)
        # Save output as column using column name
        df_sums[crp_name] = df_temp['sum']
        out_name = out_format(ifpri_type_name, irr_label, "hybas6-adm1_temp")
        out_file = os.path.join(input_folder, out_name)
#         df_sums.to_csv(out_file)
    # Save file
    out_name = out_format(ifpri_type_name, irr_label, "hybas6-adm1_add")
    print(out_name)
    out_file = os.path.join(input_folder, out_name)
#     df_sums.to_csv(out_file)
    

Irrigation Type: a 
Length: 42
maiz
crop_production_a_hybas6-adm1_add.csv


In [13]:
all_name = out_format(ifpri_type_name, irr_label, "hybas6-adm1")
all_file = os.path.join(input_folder, all_name)
df_all = pd.read_csv(all_file, header = 0, index_col = 0)

In [18]:
df_m = pd.merge(df_all, df_sums, how = 'left', left_on = 'string_id', right_on = 'string_id')
df_m.set_index(['string_id'], inplace = True)

In [19]:
df_m = df_m.reindex(sorted(df_m.columns), axis=1)

In [25]:
df_m.to_csv(all_file)

In [43]:
df_old = pd.read_csv(os.path.join(input_folder, "crops_per_watershed_i.csv"), header = 0, index_col = 0)

In [30]:
df_sums[df_sums.string_id.str.contains('USA')]

,string_id,maiz
1523,id_-9999_USA.1_1,NaN
1524,id_-9999_USA.10_1,NaN
1525,id_-9999_USA.11_1,NaN
1526,id_-9999_USA.12_1,3768.600342
1527,id_-9999_USA.13_1,NaN
...,...,...
36396,id_813909_USA.2_1,NaN
36398,id_821001_USA.2_1,NaN
36400,id_821002_USA.2_1,NaN
36403,id_821003_USA.2_1,NaN


# Ran Zonal Stats in Arc. Read in results here

In [16]:
# Read in watershed/state shapefile
poly_shp = gpd.read_file(GDBIN, layer='hybas6_adm1')
# Drop extra rows
poly = poly_shp.drop(['Shape_Leng','Shape_Length', 'Shape_Area', 'geometry'], axis = 1)
poly.set_index(['string_id'], inplace =True)

In [49]:
import fiona 
import geopandas as gpd

# Get all the layers from the .gdb file 
layers = fiona.listlayers(ifpri_gdb)
df = pd.DataFrame(index = poly.index)
for layer in layers:
    name = layer[-6:]
    print(name)
    gdf = gpd.read_file(ifpri_gdb,layer=layer)
    # Set string ID as index, only leave sum 
    df_temp = gdf.filter(['SUM', 'string_id']).set_index(['string_id'])
    df_temp.rename(columns = {"SUM": name}, inplace=True)
    df = df.join(df_temp)

MAIZ_A
BARL_A
CASS_A
CNUT_A
COCO_A
COTT_A
LENT_A
OCER_A
OFIB_A
OILP_A
OPUL_A
RAPE_A
RICE_A
SORG_A
SOYB_A
SUGC_A
SUNF_A
WHEA_A
WHEA_I
SUNF_I
SUGC_I
SOYB_I
SORG_I
RICE_I
RAPE_I
OPUL_I
OILP_I
OFIB_I
OCER_I
MAIZ_I
LENT_I
COTT_I
COCO_I
CNUT_I
CASS_I
BARL_I


In [50]:
df.sum()

MAIZ_A    8.528961e+08
BARL_A    1.357996e+08
CASS_A    2.426205e+08
CNUT_A    5.841644e+07
COCO_A    4.379385e+06
COTT_A    6.964655e+07
LENT_A    4.365591e+06
OCER_A    6.161580e+07
OFIB_A    4.856180e+06
OILP_A    2.289111e+08
OPUL_A    2.017124e+07
RAPE_A    6.241386e+07
RICE_A    7.017352e+08
SORG_A    5.807916e+07
SOYB_A    2.500704e+08
SUGC_A    1.718816e+09
SUNF_A    3.505070e+07
WHEA_A    6.746638e+08
WHEA_I    2.426637e+08
SUNF_I    1.641829e+06
SUGC_I    7.182352e+08
SOYB_I    1.325929e+07
SORG_I    1.041818e+07
RICE_I    5.432780e+08
RAPE_I    6.199013e+06
OPUL_I    2.251277e+06
OILP_I    4.496036e+05
OFIB_I    5.972498e+05
OCER_I    5.969283e+06
MAIZ_I    2.132890e+08
LENT_I    4.152511e+05
COTT_I    4.368532e+07
COCO_I    5.998070e+04
CNUT_I    5.882995e+05
CASS_I    5.359289e+06
BARL_I    1.468068e+07
dtype: float64

In [45]:
df_old.sum()

barl    1.468111e+07
coco    5.998070e+04
cott    4.367079e+07
grou    1.006144e+07
maiz    2.127878e+08
ocer    6.054274e+06
oilp    4.496036e+05
rape    6.197659e+06
sorg    1.035644e+07
soyb    1.326882e+07
sugc    7.153636e+08
sunf    1.638397e+06
whea    2.412889e+08
ofib    5.959638e+05
rice    5.430503e+08
cnut    5.813784e+05
dtype: float64

In [67]:
cargill_crops = ['barl', 'bean', 'cass', 'cnut', 'coco', 'cott', 'lent',
                 'maiz', 'ocer', 'ofib', 'oilp', 'rape', 'rice', 'sorg',
                 'soyb', 'sunf', 'trof', 'whea', 'sugc', 'opul']
df_final.filter(cargill_crops)[df_final.index.str.contains("USA")].sum()

Series([], dtype: float64)

In [44]:
df_old.sum()

barl    1.468111e+07
coco    5.998070e+04
cott    4.367079e+07
grou    1.006144e+07
maiz    2.127878e+08
ocer    6.054274e+06
oilp    4.496036e+05
rape    6.197659e+06
sorg    1.035644e+07
soyb    1.326882e+07
sugc    7.153636e+08
sunf    1.638397e+06
whea    2.412889e+08
ofib    5.959638e+05
rice    5.430503e+08
cnut    5.813784e+05
dtype: float64

In [83]:
poly[[x for x in poly if x[0:4].lower() in cargill_crops]].sum()

BARL_A    1.468068e+07
BARL_I    2.285751e+07
BEAN_A    3.003129e+06
BEAN_I    2.426205e+08
CASS_A    5.359289e+06
CASS_I    1.103945e+07
CNUT_A    5.882995e+05
CNUT_I    4.379385e+06
COCO_A    5.998070e+04
COCO_I    6.964655e+07
COTT_A    4.368532e+07
COTT_I    5.522223e+06
LENT_A    4.152511e+05
LENT_I    8.528961e+08
MAIZ_A    2.132890e+08
MAIZ_I    6.161580e+07
OCER_A    5.969283e+06
OCER_I    4.856180e+06
OFIB_A    5.972498e+05
OFIB_I    2.289111e+08
OILP_A    4.496036e+05
OILP_I    2.685075e+07
OPUL_A    2.251277e+06
OPUL_I    1.888177e+07
RAPE_A    6.199013e+06
RAPE_I    3.742856e+06
RICE_A    5.432780e+08
RICE_I    4.382200e+06
SORG_A    1.041818e+07
SORG_I    2.500704e+08
SOYB_A    1.325929e+07
SOYB_I    2.452269e+08
SUGC_A    7.182352e+08
SUGC_I    3.505070e+07
SUNF_A    1.641829e+06
SUNF_I    1.043619e+08
TROF_A    6.239887e+07
TROF_I    9.192818e+08
WHEA_A    2.426637e+08
WHEA_I    5.245485e+07
dtype: float64

In [84]:
poly

,PFAF_ID,GID_1,GID_0,ACOF_A,ACOF_I,BANA_A,BANA_I,BARL_A,BARL_I,BEAN_A,...,TEMF_I,TOBA_A,TOBA_I,TROF_A,TROF_I,VEGE_A,VEGE_I,WHEA_A,WHEA_I,YAMS_A
string_id,,,,,,,,,,,,,,,,,,,,,
id_-9999_AGO.1_1,-9999.0,AGO.1_1,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_-9999_AGO.11_1,-9999.0,AGO.11_1,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_-9999_AGO.16_1,-9999.0,AGO.16_1,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_-9999_AGO.18_1,-9999.0,AGO.18_1,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_-9999_AGO.2_1,-9999.0,AGO.2_1,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_914807_GRL.2_1,914807.0,GRL.2_1,GRL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_914808_GRL.2_1,914808.0,GRL.2_1,GRL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_914809_GRL.2_1,914809.0,GRL.2_1,GRL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df = df.reindex(sorted(df.columns), axis=1)
crop_lookup_path = os.path.join(input_folder, "ifpri_production.csv")
df.to_csv(crop_lookup_path)

In [7]:
poly

,ACOF_A,ACOF_I,BANA_A,BANA_I,BARL_A,BARL_I,BARL_R,BEAN_A,BEAN_I,BEAN_R,...,TEMF_I,TOBA_A,TOBA_I,TROF_A,TROF_I,VEGE_A,VEGE_I,WHEA_A,WHEA_I,YAMS_A
string_id,,,,,,,,,,,,,,,,,,,,,
id_-9999_AGO.1_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_-9999_AGO.11_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_-9999_AGO.16_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_-9999_AGO.18_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_-9999_AGO.2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_914807_GRL.2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_914808_GRL.2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id_914809_GRL.2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# WFN

In [184]:
df_c = pd.read_csv(crop_lookup_path, header = 0, index_col = None)
wfn_codes = df_c.WFN.tolist()
df_crp = df_c.filter(['WFN', 'SPAM_code']).set_index(['WFN'])

df_st = pd.read_csv(os.path.join(root, "WRI_Data", "sources", "WFN", "state_to_gadm.csv"))
df_st['location'] = df_st['State'] + ", " + df_st['Country']

# Water Footprint Network
df_wfn = pd.read_excel(os.path.join(root, "WRI_Data", "sources", "WFN", "WFN_state_stats.xlsx"), header=None, index_col = None)
# Clean water footprint
df_wfn = df_wfn[3:] # Clean. drop the first 3 rows (no data)
df_wfn.iloc[:,0:7].fillna(method = 'ffill', inplace=True) # Clean. forward fill all crop info so every row associated with right crop

# Filter out crops
df_filt = df_wfn[df_wfn[3].isin(wfn_codes)]
# Create unique WFN code name
df_filt2 = pd.merge(df_filt, df_crp, how = 'left', left_on = 3, right_index = True)
df_filt2['wfn_code'] = df_filt2['SPAM_code'] + "_" + df_filt2[8] 
df_filt2.set_index(['wfn_code'], inplace = True)
# Flip data for locations are rows and WFN are columns
df_w = df_filt2.iloc[:, 10:-1].transpose()
# Pull out state & country names, find GIDs 
df_l = df_wfn.iloc[0:2, ].transpose()
df_l = df_l.iloc[10:,]
df_l['location'] = df_l[4] + ", " + df_l[3]
df_l.reset_index(inplace = True)
df_names = pd.merge(df_l, df_st, how = 'left', left_on = 'location', right_on = 'location')
# Merge GIDs to final WFN data
df_wfn_final = pd.merge(df_w, df_names, how = 'left', left_index = True, right_on = 'index')
# Clean up WFN data
df_wfn_final.drop(['index', 3, 4, 'Country', 'State'], axis = 1, inplace=True)
df_wfn_final.GID_1[df_wfn_final.GID_1 == 'avg'] = df_wfn_final.GID_0 + "_avg"
df_wfn_final.GID_1[df_wfn_final.location.str.contains("Global")] = "Global"

# Create dataframe to hold all values
df_final = pd.DataFrame(index = df_wfn_final.columns)
# Read in all State GIDs
df_gids = pd.read_csv(os.path.join(root, "WRI_Data", "sources", "WFN", "gadm36_1.csv"))
# Create a list of country and state IDs
gid0s = sorted(list(set(df_gids.GID_0.tolist())))
gid1s = sorted(set(df_gids.GID_1.tolist()))

df_glob = df_wfn_final[df_wfn_final.GID_1 == "Global"].max().to_frame()

for z in tqdm(range(len(gid0s))):
    gid0 = gid0s[z]
    gid0_gid1s = [x for x in gid1s if gid0 in x] # States within country
    # Find Country Averge
    df_gid0 = df_wfn_final[df_wfn_final.GID_1 == gid0 + "_avg"].max().to_frame()
    # Replace nulls in country data with global data
    df_gid0.update(df_glob, overwrite = False)
    
    # Now loop through states, replacing null data with country averages
    for i in range(len(gid0_gid1s)): 
        gid1 = gid0_gid1s[i]
        # Find max WFN values for state
        df_gid1 = df_wfn_final[df_wfn_final.GID_1 == gid1].max().to_frame()
        # Replace nulls in state data with country data
        df_gid1.update(df_gid0, overwrite = False)
        # Name using GID1 
        df_gid1.columns = [gid1]
        # Add to master
        df_final = df_final.join(df_gid1)

df_gids_final = df_final.iloc[0:-4].transpose()
wfn_lookup_path = os.path.join(input_folder, "_update_wfn_states.csv")
df_gids_final.to_csv(wfn_lookup_path)

C:\Users\samantha.kuzma\Anaconda3\envs\py_38\lib\site-packages\pandas\core\frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
100%|██████████| 228/228 [02:28<00:00,  1.53it/s]
